# Import

In [33]:
import sys
%matplotlib inline
SCRIPTS_PATH = '../scripts'
if SCRIPTS_PATH not in sys.path:
    sys.path.append(SCRIPTS_PATH)

import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import regression_model
import NNModel
from tagging import *
from NNModel import *
from window_aggregation import *
from dull_model import *

# Load test data

In [3]:
test_df = pd.read_csv('../data/test.csv')

In [4]:
full_df = pd.read_csv('../data/dataset1_v2.csv')

In [43]:
prods = full_df['PRODUCT'].unique().tolist()
selected_prods = np.random.choice(prods, 30)
out_df = full_df[full_df['PRODUCT'].isin(selected_prods)]
out_df.to_csv('../data/test.csv', index=False)

# Init models

In [5]:
nn_model = nn_init('../model/')

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/base.py:312: UserWarning: Trying to unpickle estimator OneHotEncoder from version 0.18.1 when using version 0.19.0. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [6]:
tag_model = tag_init('../model/dn/')

In [34]:
dull_model = DullModel('../model/rm/')

# Predict

In [45]:
products = test_df['PRODUCT'].unique()
alphabet = 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя '
for product in products:
    comments = test_df[test_df['PRODUCT'] == product]['TEXT'].tolist()
    prod_info = test_df[test_df['PRODUCT'] == product].head(1)
    prod_name = prod_info['NAME'].tolist()[0].strip() if len(prod_info['NAME']) > 0 else ''
    print('Продукт %d, %s' % (product, prod_name))
    
    # DN - tags
    num_chosen_tags = 3
    single_prod_df = full_df[full_df['PRODUCT'] == product]
    tags = tag_model.predict(single_prod_df)[:num_chosen_tags]
    tags = [t for t in tags if len(t.split(' ')) <=3]
    tag_line = ''
    for tag in tags:
        tag_flt = ''.join([c for c in tag.lower() if c in alphabet])
        tag_line += '#%s ' % tag_flt.replace(' ', '_')
    print('\t' + tag_line)
    
    # nn 
    wnd_agg = aggregate(comments, num_comments=3, model=nn_model, wnd_l=12, stride=6, positive_q=(0.8, 0.95), negative_threshold=3)
    print('\tПокупатели отмечают такие положительные моменты: ')
    if len(wnd_agg['+']) == 0:
        print('\t\t---')
    else:
        for com in wnd_agg['+']:
            print('\t\t...%s...' % com)
            
    print('\tПокупатели отмечают такие отрицательные моменты: ')
    if len(wnd_agg['-']) == 0:
        print('\t\t---')
    else:
        for com in wnd_agg['-']:
            print('\t\t...%s...' % com)
    
    # comments
    num_chosen_comments = 3
    dull_pres = dull_model.predict(single_prod_df, list(range(len(single_prod_df))))[:num_chosen_comments]
    print('\tПокупатели также находят важным: ')
    if len(dull_pres) == 0:
        print('\t\t---')
    else:
        for com in dull_pres:
            print('\t\t...%s...' % com)
    
    # coallocations
    
    
    
    print(' ')        
    print(''.join(['*'] * 110))
    print(' ')

Продукт 10000838, Автомобильная магнитола с CD MP3 Pioneer DEH-9300SD
	#плюсы_качество_изготовления 
	Покупатели отмечают такие положительные моменты: 
		...выглядит очень стильно и добротно ...
		...читает, что не может не радовать) главное прикупить пульт и про якобы ...
		...аппарат отличный ...
	Покупатели отмечают такие отрицательные моменты: 
		---
	Покупатели также находят важным: 
		...представитель  через  него  подключены  усилитель  на  четыре  колонки  и  сабвуфер  аппарат  отличный  а...
		...этот  аппарат  на  достойном  уровне...
		...стоит  штатная  акустика  пионер  саба  нет  не  люблю  глухой  бас  короче  звук  отличный...
 
**************************************************************************************************************
 
Продукт 10001253, Телевизор LG 32 LW4500
	#изображение_отличное #_изображение_хорошее #пульт_удобный 
	Покупатели отмечают такие положительные моменты: 
		...изображение отличное ...
		...настройки простые и удобные настроил тв не отк

	#тепловентилятор_очень_хороший 
	Покупатели отмечают такие положительные моменты: 
		...тепловентилятор очень хороший, есть три режима - холодный воздух (на лето), теплый ...
		...я лично даже не обращаю на него внимания ...
		...дома тоже пару раз включали, но тут обязательно надо позаботиться об увлажнении ...
	Покупатели отмечают такие отрицательные моменты: 
		---
	Покупатели также находят важным: 
		...регулятор  на  максимум  правый  включить  обогрев  после  того  как  почувствуете  что  температура  комфортна...
		...так  как  ожидались  морозы  до  с  а  в  доме  было  уже  прохладно  и...
		...быстро  расплавился...
 
**************************************************************************************************************
 
Продукт 20032782, Мультиварка Tefal RK812832
	#брал_по_акции #вполне_доволен_покупкой 
	Покупатели отмечают такие положительные моменты: 
		...отлично вписалась в мою кухню, надеюсь так же отлично будет готовить ...
		...для людей, стоящих перед выбо

	Покупатели отмечают такие положительные моменты: 
		...на глаза мне попал thrustmaster gpx lightback ferrari , яркая упаковка ...
		...присутствует , форма прежняя, как и на классическом удобно вливается в руки ...
	Покупатели отмечают такие отрицательные моменты: 
		---
	Покупатели также находят важным: 
		...успел  выкинуть  выбор  пал  именно  на  него  поначалу  привлекает  внимание  светодиодов  отображающие  процессы...
 
**************************************************************************************************************
 
Продукт 40064074, Видеоигра для PS4 . Pro Evolution Soccer 2016
	#отличная_игра_физика 
	Покупатели отмечают такие положительные моменты: 
		...игра лучшая в своём жанре ...
	Покупатели отмечают такие отрицательные моменты: 
		---
	Покупатели также находят важным: 
		...лучшей  в  своем  жанре...
		...похожи  на  реальность  атмосфера  игры  завораживает  празднование  выигранного  титула  забитого  мяча  горечь  поражения...
		...популярным  футболь